In [2]:
import os
import sys
import gdal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from math import isclose
import math        


import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from itertools import product        
from tqdm import tqdm_notebook

import warnings                                 
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
import xgboost as xgb

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

In [ ]:
%matplotlib inline

In [ ]:
"""
from simpledbf import Dbf5

dbf = Dbf5('FireAnomalies.dbf')

df = dbf.to_dataframe()"""

In [ ]:
fp = r'C:\Users\Miguel\Desktop\Teledeteccion\Anomalias/fire_archive_M6_80980.shp'

df = gpd.read_file(fp)

df.head()

In [ ]:
df['Year'] = pd.to_datetime(df['ACQ_DATE']).dt.year

In [ ]:
new_df = df[(df['CONFIDENCE'] > 50) & (df['Year'] == 2001)]

In [ ]:
new_df.head()

In [ ]:
list_years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [3]:
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona
from plotly import graph_objs as go

In [ ]:
# Generamos la figura del mapa
fig = go.Figure()
 
# Recorremos los datos del archivo csv y extraemos sus coordenadas,
# etiquetas a mostrar y especificamos características del marcador
for i in new_df:
    fig.add_trace(go.Scattergeo(
            lon = new_df['LONGITUDE'],
            lat = new_df['LATITUDE'],
            text = new_df['Year'],
            marker = dict(
                size = new_df['CONFIDENCE']*10,
                color = 'lightseagreen',
                line_color='black',
                line_width=0.5,
                sizemode = 'area'
        )))
 

fig.layout.update(
    title_text = 'Población en las ciudades del mundo, año 2025',
    showlegend = False,
    geo = dict(
        scope='europe',
        resolution=110,
        projection_type='mollweide',
        showcoastlines=True,
        showocean=True,
        oceancolor='#132630',
        lakecolor='#132630',
        coastlinecolor='#224354',
        landcolor='#224354',
    )
)
 
fig.show()

In [ ]:
for year in list_years:
    
    df_year = new_df[new_df['Year'] == year]
    
    points = [Point(x, y) for x, y in zip(df_year.LONGITUDE, df_year.LATITUDE)]
    
    gdf = gpd.GeoDataFrame(df_year, geometry=points)
    
    schema = gpd.io.file.infer_schema(gdf)
    
    schema['properties']['Year'] = 'int'
    
    gdf.crs = {'init' :'epsg:4326'}

    gdf.to_file(driver='ESRI Shapefile', filename='Anomalias_'+str(year)+'.shp', schema=schema)

In [ ]:
fp = r'C:\Users\Miguel\Desktop\Teledeteccion\MFE_50/MFE50CAT.shp'

df = gpd.read_file(fp)

df.head()

In [ ]:
df.USOS_SUELO.unique()

In [ ]:
usos_ajenos = ['Agua','Artificial','Cultivos']

In [ ]:
for uso in usos_ajenos:

    df_forestal = df.drop(df[df.USOS_SUELO == uso].index)

In [ ]:
df_forestal.USOS_SUELO.unique()

In [ ]:
df_forestal.columns

In [ ]:
important_columns = ['POLIGON', 'PROV_MFE50', 'CCAA_MFE50', 'TFCCTOT', 'TFCCARB', 'USOS_SUELO', 'Shape_Leng', 'Shape_Area',\
       'geometry']


In [ ]:
for col in df_forestal.columns:
    
    if col not in important_columns:
        
        df_forestal = df_forestal.drop(col, axis=1)

In [ ]:
df_forestal.head()

In [ ]:
df_forestal.crs

In [ ]:
df_forestal.to_file(driver='ESRI Shapefile', filename='cat_forestal.shp')

In [4]:
fp = r'C:\Users\Miguel\Desktop\Teledeteccion\MFE_50/MFECAT_DIS_31.shp'

mfe_dis = gpd.read_file(fp)

mfe_dis.head()

,CCAA_MFE50,Dis,geometry
0,51,0.0,(POLYGON ((284164.1827030445 4491033.087758471...
1,51,0.0,"POLYGON ((493174.6282663728 4682855.995075431,..."


In [8]:
type(mfe_dis.geometry[0])

shapely.geometry.multipolygon.MultiPolygon

In [ ]:
mfe_dis.geometry

In [ ]:
list_years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2011,2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [1]:
fp_inc = r'C:\Users\Miguel\Desktop\Teledeteccion\IncendisCat/incendis10.shp'

incendis = gpd.read_file(fp_inc)

NameError: name 'gpd' is not defined

In [ ]:
inter = gpd.overlay(mfe_dis, incendis, how='intersection')
inter = inter.dissolve(by='CODI_FINAL', aggfunc='first')

In [ ]:
inter.head()

In [ ]:
inter.to_file(driver='ESRI Shapefile', filename='incmfe_2010.shp')

In [ ]:
for year in list_years:
    
    fp_inc = r'C:\Users\Miguel\Desktop\Teledeteccion\IncendisCat/incendis'+str(year)+'.shp'
    
    incendis = gpd.read_file(fp_inc)
    
    inter = gpd.overlay(mfe_dis, incendis, how='intersection')
    
    inter = inter.dissolve(by='CODI_FINAL', aggfunc='first')
    
    inter.to_file(driver='ESRI Shapefile', filename='incmfe_'+str(year)+'.shp')